My notes:

- finetuning model to use: facebook/wav2vec2-base on huggingface
- using voice recording app on iphone to record audio clips
- organize audio into clips by command
-** project concept:**
-** Commands to use: **
-

In [1]:
!pip install pydub transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [2]:
import os
from pydub import AudioSegment
from datasets import load_dataset
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification, TrainingArguments, Trainer

In [18]:
from google.colab import files
uploaded = files.upload()

Saving New Recording 1.m4a to New Recording 1.m4a
Saving New Recording 2.m4a to New Recording 2.m4a
Saving New Recording 3.m4a to New Recording 3.m4a
Saving New Recording 4.m4a to New Recording 4.m4a
Saving New Recording 5.m4a to New Recording 5.m4a
Saving New Recording 6.m4a to New Recording 6.m4a
Saving New Recording 7.m4a to New Recording 7.m4a
Saving New Recording 8.m4a to New Recording 8.m4a
Saving New Recording 9.m4a to New Recording 9.m4a
Saving New Recording 10.m4a to New Recording 10.m4a
Saving New Recording 11.m4a to New Recording 11.m4a
Saving New Recording 12.m4a to New Recording 12.m4a
Saving New Recording 13.m4a to New Recording 13.m4a
Saving New Recording 14.m4a to New Recording 14.m4a
Saving New Recording 15.m4a to New Recording 15.m4a
Saving New Recording 16.m4a to New Recording 16.m4a
Saving New Recording 17.m4a to New Recording 17.m4a
Saving New Recording 18.m4a to New Recording 18.m4a
Saving New Recording 19.m4a to New Recording 19.m4a
Saving New Recording 20.m4a to

In [20]:
import os
import shutil

# Create directories
m4a_dir = "/content/LL_final_project"
os.makedirs(f"{m4a_dir}/LL Final Project NextStep", exist_ok=True)
os.makedirs(f"{m4a_dir}/LL FinalProject StartRecipe", exist_ok=True)

# Move files to respective folders
for i in range(1, 21):
    file_name = f"New Recording {i}.m4a"
    if os.path.exists(file_name):
        if i <= 10:  # 1-10 for startrecipe
            shutil.move(file_name, f"{m4a_dir}/LL FinalProject StartRecipe/{file_name}")
        else:  # 11-20 for nextstep
            shutil.move(file_name, f"{m4a_dir}/LL Final Project NextStep/{file_name}")

# Verify the structure
!ls /content/LL_final_project/*

'/content/LL_final_project/LL Final Project NextStep':
'New Recording 11.m4a'	'New Recording 14.m4a'	'New Recording 17.m4a'	'New Recording 20.m4a'
'New Recording 12.m4a'	'New Recording 15.m4a'	'New Recording 18.m4a'
'New Recording 13.m4a'	'New Recording 16.m4a'	'New Recording 19.m4a'

'/content/LL_final_project/LL FinalProject StartRecipe':
'New Recording 10.m4a'	'New Recording 3.m4a'  'New Recording 6.m4a'  'New Recording 9.m4a'
'New Recording 1.m4a'	'New Recording 4.m4a'  'New Recording 7.m4a'
'New Recording 2.m4a'	'New Recording 5.m4a'  'New Recording 8.m4a'


In [21]:
# Convert M4A to WAV and organize into folders
def convert_m4a_to_wav(m4a_dir, data_dir):
    folder_to_label = {
        "LL Final Project NextStep": "nextstep",
        "LL FinalProject StartRecipe": "startrecipe"
    }

    for folder_name, label in folder_to_label.items():
        command_dir = os.path.join(data_dir, label)
        os.makedirs(command_dir, exist_ok=True)
        folder_path = os.path.join(m4a_dir, folder_name)

        for i, filename in enumerate(os.listdir(folder_path), 1):
            if filename.endswith(".m4a"):
                m4a_path = os.path.join(folder_path, filename)
                wav_filename = f"{label}_{i:02d}.wav"
                wav_path = os.path.join(command_dir, wav_filename)
                audio = AudioSegment.from_file(m4a_path, format="m4a")
                audio = audio.set_frame_rate(16000)
                audio.export(wav_path, format="wav")

In [22]:
# Load the dataset
def load_audio_dataset(data_dir):
    dataset = load_dataset("audiofolder", data_dir=data_dir)
    return dataset

In [23]:
# Preprocess audio data
def preprocess_audio(dataset):
    feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")
    def preprocess_function(examples):
        audio_arrays = [x["array"] for x in examples["audio"]]
        inputs = feature_extractor(
            audio_arrays,
            sampling_rate=16000,
            max_length=16000,
            truncation=True
        )
        return inputs
    encoded_dataset = dataset.map(preprocess_function, remove_columns=["audio"], batched=True)
    return encoded_dataset

In [24]:
# Step 4: Load model
def load_model():
    model = AutoModelForAudioClassification.from_pretrained(
        "facebook/wav2vec2-base",
        num_labels=2
    )
    return model

In [26]:
# Set up and run training
def train_model(encoded_dataset, model):
    small_dataset = encoded_dataset["train"].train_test_split(test_size=0.2)
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=1,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        logging_steps=1,
        learning_rate=3e-5,
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=small_dataset["train"],
        eval_dataset=small_dataset["test"],
    )
    trainer.train()

def main():
    m4a_dir = "/content/LL_final_project"
    data_dir = "/content/data"

    print("Converting M4A files to WAV...")
    convert_m4a_to_wav(m4a_dir, data_dir)

    print("Loading dataset...")
    dataset = load_audio_dataset(data_dir)
    print("Preprocessing audio...")
    encoded_dataset = preprocess_audio(dataset)

    print("Loading Wav2Vec2 model...")
    model = load_model()

    print("Starting compatibility test...")
    train_model(encoded_dataset, model)
    print("Training completed successfully!")

if __name__ == "__main__":
    main()

Converting M4A files to WAV...
Loading dataset...


Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Preprocessing audio...


/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:315: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


...

Loading Wav2Vec2 model...


Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting compatibility test...


Step,Training Loss
1,0.710500
2,0.665800
3,0.764400
4,0.657800


Training completed successfully!
